In [32]:
import numpy as np
import pandas as pd;
import matplotlib.pyplot as plt;
import seaborn as sns;
from sklearn.impute import SimpleImputer;
from sklearn.compose import ColumnTransformer;
from sklearn.pipeline import Pipeline;
from sklearn.preprocessing import LabelEncoder;
from sklearn.preprocessing import StandardScaler;
from sklearn.preprocessing import MinMaxScaler;
from sklearn.model_selection import train_test_split;
from sklearn.linear_model import LinearRegression ;
from sklearn.linear_model import LogisticRegression;
from sklearn.linear_model import Ridge, Lasso;
from sklearn.metrics import mean_squared_error;
from sklearn.metrics import r2_score;
from sklearn.preprocessing import PolynomialFeatures;
from sklearn.svm import SVR;
from sklearn.svm import SVC;
from sklearn.tree import DecisionTreeClassifier;
from sklearn.tree import DecisionTreeRegressor;
from sklearn.ensemble import RandomForestClassifier;
from sklearn.ensemble import RandomForestRegressor;
from sklearn.neighbors import KNeighborsClassifier;
from sklearn.neighbors import KNeighborsRegressor;
from sklearn.naive_bayes import GaussianNB;
import xgboost as xgb;
from xgboost import XGBClassifier;
from xgboost import XGBRegressor;

import tensorflow as tf
import keras;
from keras_preprocessing import image;
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam;
from keras.callbacks import ModelCheckpoint;
from keras.models import Sequential;
from tensorflow.keras.applications import VGG16;
from tensorflow.keras.applications import InceptionResNetV2;
from keras.applications.vgg16 import preprocess_input;
from tensorflow.keras.applications.vgg16 import decode_predictions;
from tensorflow.keras.callbacks import EarlyStopping;

import os;
from os import listdir;
from PIL import Image as PImage;
import cv2

In [33]:
from tqdm import tqdm

We have imported the libraries. 

Its Time to export the data

# Data

In [34]:
train = pd.read_csv('../input/identify-snake-breed-hackerearth/dataset/train.csv');
test = pd.read_csv('../input/identify-snake-breed-hackerearth/dataset/test.csv')

# How many types of breeds are there?

In [35]:
train['breed'].unique()

array(['nerodia-erythrogaster', 'pantherophis-vulpinus',
       'thamnophis-sirtalis', 'pantherophis-obsoletus',
       'agkistrodon-contortrix', 'crotalus-atrox',
       'lampropeltis-triangulum', 'crotalus-horridus', 'crotalus-ruber',
       'heterodon-platirhinos', 'nerodia-sipedon', 'thamnophis-elegans',
       'thamnophis-marcianus', 'crotalus-viridis', 'nerodia-fasciata',
       'haldea-striatula', 'storeria-dekayi', 'agkistrodon-piscivorus',
       'nerodia-rhombifer', 'storeria-occipitomaculata',
       'thamnophis-radix', 'coluber-constrictor', 'natrix-natrix',
       'diadophis-punctatus', 'masticophis-flagellum',
       'pantherophis-spiloides', 'rhinocheilus-lecontei',
       'lampropeltis-californiae', 'pituophis-catenifer',
       'opheodrys-aestivus', 'pantherophis-guttatus',
       'pantherophis-alleghaniensis', 'thamnophis-proximus',
       'pantherophis-emoryi', 'crotalus-scutulatus'], dtype=object)

In [36]:
train.head()

,image_id,breed
0,a8b3ad1dde,nerodia-erythrogaster
1,8b492b973d,pantherophis-vulpinus
2,929b99ea92,thamnophis-sirtalis
3,bbac7385e2,pantherophis-obsoletus
4,ef776b1488,agkistrodon-contortrix


In [37]:
test.head()

,image_id
0,7ede553357
1,3500b219e4
2,d43a78d6d9
3,1f36f26994
4,8cb85fc58e


# X

In [38]:
test.shape

(2361, 1)

We have images in train folder. And, we have name of those breeds in the csv format. We have to merge between these two to make the machine understand which image belongs to which breed.

Lets run a simple for loop

In [39]:
img_width = 150;
img_height = 150;
x=[];

for i in tqdm (range(train.shape[0])):
    path = '../input/identify-snake-breed-hackerearth/dataset/train/' + train['image_id'][i] + '.jpg'
    #print(path)
    img = image.load_img(path, target_size = (img_width, img_height, 3))
    img = image.img_to_array(img)
    img = img/255.0
    x.append(img)
    
    
#x = np.array(x);
#rm path


100%|██████████| 5508/5508 [00:07<00:00, 762.17it/s]


Nice, the images are extracted. We have got our x. Now we need y

In [40]:
 y = train.drop(columns = ['image_id'], axis = 1)

In [41]:
y = pd.get_dummies(y)

In [42]:
y = y.to_numpy()

In [43]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

Nice. So, we have converted our y file into numpy array. But how about x? Shouldn't it be a numpy array also?

In [44]:
x = np.array(x)

# Simple Train Test Split

In [46]:
mtrain, mtest, ntrain, ntest = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [47]:
mtrain.shape, ntrain.shape

((4406, 150, 150, 3), (4406, 35))

# Deep Model

In [48]:
mod = keras.models.Sequential([
                        keras.layers.Conv2D(16, (3,3), activation = 'relu', input_shape = mtrain[0].shape),
                        keras.layers.BatchNormalization(),
                        keras.layers.MaxPooling2D(pool_size=(2,2)),
                        keras.layers.Dropout(0.30),
    
                        keras.layers.Conv2D(16, (3,3), activation = 'relu'),
                        keras.layers.BatchNormalization(),
                        keras.layers.MaxPooling2D(pool_size=(2,2)),
                        keras.layers.Dropout(0.30),
    
                        keras.layers.Conv2D(16, (3,3), activation = 'relu'),
                        keras.layers.BatchNormalization(),
                        keras.layers.MaxPooling2D(pool_size=(2,2)),
                        keras.layers.Dropout(0.40),
    
                        keras.layers.Conv2D(16, (3,3), activation = 'relu'),
                        keras.layers.BatchNormalization(),
                        keras.layers.MaxPooling2D(pool_size=(2,2)),
                        keras.layers.Dropout(0.50),
    
                        keras.layers.Flatten(),
     
                        keras.layers.Dense(units = 128, activation = 'relu'),
                        keras.layers.BatchNormalization(),
                        keras.layers.Dropout(0.50),
    
                        keras.layers.Dense(units = 128, activation = 'relu'),
                        keras.layers.BatchNormalization(),
                        keras.layers.Dropout(0.50),
    
    
                        keras.layers.Dense(units = 35, activation = 'softmax')
])
                        

In [49]:
mod.compile(optimizer= 'Adam' , loss = 'categorical_crossentropy' , metrics= ['accuracy'])

# Training

In [50]:
mod.fit(mtrain, ntrain, epochs= 100, batch_size = 64, validation_data = (mtest, ntest))

Epoch 1/100
69/69 [==============================] - 2s 30ms/step - loss: 4.6494 - accuracy: 0.0352 - val_loss: 3.6311 - val_accuracy: 0.0617
Epoch 2/100
69/69 [==============================] - 2s 26ms/step - loss: 4.2264 - accuracy: 0.0445 - val_loss: 3.8870 - val_accuracy: 0.0617
Epoch 3/100
69/69 [==============================] - 2s 27ms/step - loss: 3.9460 - accuracy: 0.0592 - val_loss: 4.1014 - val_accuracy: 0.0617
Epoch 4/100
69/69 [==============================] - 2s 25ms/step - loss: 3.8021 - accuracy: 0.0676 - val_loss: 4.1792 - val_accuracy: 0.0626
Epoch 5/100
69/69 [==============================] - 2s 26ms/step - loss: 3.6999 - accuracy: 0.0713 - val_loss: 4.1385 - val_accuracy: 0.0626
Epoch 6/100
69/69 [==============================] - 2s 25ms/step - loss: 3.6306 - accuracy: 0.0849 - val_loss: 4.0116 - val_accuracy: 0.0590
Epoch 7/100
69/69 [==============================] - 2s 25ms/step - loss: 3.5792 - accuracy: 0.0792 - val_loss: 3.8932 - val_accuracy: 0.0690
Epoch 

# Test Data

In [51]:
img_width = 150;
img_height = 150;
z=[];

for i in tqdm (range(test.shape[0])):
    pathe = '../input/identify-snake-breed-hackerearth/dataset/test/' + test['image_id'][i] + '.jpg'
    img1 = image.load_img(pathe, target_size = (img_width, img_height, 3))
    img1 = image.img_to_array(img1)
    img1 = img1/255.0
    #img1 = img1.reshape(1,img_width, img_height, 3 )
    z.append(img1)

100%|██████████| 2361/2361 [00:04<00:00, 493.20it/s]


Let's convert it into a numpy array

In [53]:
z = np.array(z)

# Pridiction

In [55]:
res = mod.predict(z)

In [56]:
res = np.array(res)

# Getting the Submission File Ready

In [57]:
cl = train['breed'].unique()
cl

array(['nerodia-erythrogaster', 'pantherophis-vulpinus',
       'thamnophis-sirtalis', 'pantherophis-obsoletus',
       'agkistrodon-contortrix', 'crotalus-atrox',
       'lampropeltis-triangulum', 'crotalus-horridus', 'crotalus-ruber',
       'heterodon-platirhinos', 'nerodia-sipedon', 'thamnophis-elegans',
       'thamnophis-marcianus', 'crotalus-viridis', 'nerodia-fasciata',
       'haldea-striatula', 'storeria-dekayi', 'agkistrodon-piscivorus',
       'nerodia-rhombifer', 'storeria-occipitomaculata',
       'thamnophis-radix', 'coluber-constrictor', 'natrix-natrix',
       'diadophis-punctatus', 'masticophis-flagellum',
       'pantherophis-spiloides', 'rhinocheilus-lecontei',
       'lampropeltis-californiae', 'pituophis-catenifer',
       'opheodrys-aestivus', 'pantherophis-guttatus',
       'pantherophis-alleghaniensis', 'thamnophis-proximus',
       'pantherophis-emoryi', 'crotalus-scutulatus'], dtype=object)

In [58]:
c = res.argmax(axis = 1);
c
#c.shape

array([18,  3,  3, ..., 28,  3, 18])

In [59]:
ped = [cl [index] for index in c];
hehe = pd.DataFrame({'breed': ped})

In [60]:
hehe.set_index('breed').to_csv('submissionnew.csv')